# FESTA Demo - LLaVA 1.6 7B Testing

Simple notebook to test LLaVA 1.6 7B model with FESTA example images.

In [ ]:
# Install required packages
!pip install torch torchvision transformers pillow accelerate bitsandbytes
!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git

In [ ]:
import torch
import json
import requests
from PIL import Image
import matplotlib.pyplot as plt
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Load LLaVA 1.6 7B model
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
    device_map="auto"
)

print("LLaVA 1.6 7B model loaded successfully!")

In [ ]:
# GitHub base URL for examples
base_url = "https://raw.githubusercontent.com/iiscleap/mllm-uncertainty-estimation/main/examples/"

def load_image_from_url(image_name):
    url = base_url + image_name
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def generate_response(image, question):
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    
    inputs = processor(prompt, image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7
        )
    
    response = processor.decode(output[0], skip_special_tokens=True)
    response = response.split("ASSISTANT:")[-1].strip()
    
    return response

def test_example(image_name, question, title):
    image = load_image_from_url(image_name)
    response = generate_response(image, question)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{title}\nQuestion: {question}\nAnswer: {response}", fontsize=12, pad=20)
    plt.tight_layout()
    plt.show()
    
    return response

## Test with 6 FESTA Examples

In [ ]:
# Example 1: Original Spatial Relation
response = test_example(
    "val_Spatial_Relation_1.jpg",
    "Is the car beneath the cat?",
    "Example 1: Original Spatial Relation"
)
print(f"Response: {response}\n")

In [ ]:
# Example 2: Contrast Perturbation
response = test_example(
    "val_Spatial_Relation_1_contrast1.jpg",
    "Is the car beneath the cat?",
    "Example 2: Contrast Perturbation"
)
print(f"Response: {response}\n")

In [ ]:
# Example 3: Masking Perturbation
response = test_example(
    "val_Spatial_Relation_1_masking1.jpg",
    "Is the car beneath the cat?",
    "Example 3: Masking Perturbation"
)
print(f"Response: {response}\n")

In [ ]:
# Example 4: Negated/Complementary Version
response = test_example(
    "val_Spatial_Relation_1_negated_contrast1.jpg",
    "Is the car beneath the cat?",
    "Example 4: Negated/Complementary Version"
)
print(f"Response: {response}\n")

In [ ]:
# Example 5: Different Scene Original
response = test_example(
    "val_Spatial_Relation_5.jpg",
    "Are there animals in this image?",
    "Example 5: Different Scene Original"
)
print(f"Response: {response}\n")

In [ ]:
# Example 6: Different Scene Blur
response = test_example(
    "val_Spatial_Relation_5_blur1.jpg",
    "Are there animals in this image?",
    "Example 6: Different Scene Blur"
)
print(f"Response: {response}\n")

In [ ]:
# Custom testing function - modify as needed
def test_custom():
    # Change these values to test other images/questions
    image_name = "val_Spatial_Relation_1.jpg"  # Change this
    question = "What do you see in this image?"  # Change this
    
    response = test_example(image_name, question, "Custom Test")
    print(f"Custom Response: {response}")

# Uncomment to use custom testing
# test_custom()